In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat


In [6]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian"
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon ginger, minced
- 1 can (14 oz) tomato sauce
- 1 tablespoon garam masala
- 1 teaspoon ground cumin
- 1 teaspoon ground coriander
- 1 teaspoon paprika
- 1/2 teaspoon turmeric
- Salt and pepper to taste
- Fresh cilantro, chopped (for garnish)
- Cooked rice or naan bread (for serving)

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced garlic, half of the minced ginger, and a pinch of salt. Add the chicken pieces and mix well to coat. Cover and marinate in the refrigerator for at least 1 hour, or overnight for bes

AIMessageChunk(content="As a vegetarian chef specialized in making traditional recipes vegetarian, I can offer you a delicious alternative to the Chicken Tikka Masala recipe using paneer instead of chicken. Paneer is a popular Indian cheese that is firm in texture and absorbs flavors well, making it a great substitute in this dish.\n\nHere's how you can modify the recipe to make Paneer Tikka Masala:\n\nIngredients:\n- 1 lb paneer, cut into bite-sized cubes\n- 1 cup plain yogurt\n- 2 tablespoons lemon juice\n- 2 tablespoons vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1 tablespoon ginger, minced\n- 1 can (14 oz) tomato sauce\n- 1 tablespoon garam masala\n- 1 teaspoon ground cumin\n- 1 teaspoon ground coriander\n- 1 teaspoon paprika\n- 1/2 teaspoon turmeric\n- Salt and pepper to taste\n- Fresh cilantro, chopped (for garnish)\n- Cooked rice or naan bread (for serving)\n\nInstructions:\n1. Follow the same marinating process as the original recipe, but substitute t

In [2]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self,text):
        items = text.strip().split(",")
        return list(map(str.strip,items))
    
p = CommaOutputParser()

p.parse("Hello, how, are,you")

['Hello', 'how', 'are', 'you']

In [9]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}"),
    ]
)

prompt = template.format_messages(
    max_items=10,
    question="What are the colors?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}"),
    ]
)

chain = template | chat | CommaOutputParser()

chain.invoke(
    {
    "max_items":5,
    "question":"What are the pokemons?"
    }
)




['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [5]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Hola, soy {name}!"),
        ("human", "What is distant between {country_a} and {country_b}. Also, what is your name?"),
    ]
)

prompt = template.format_messages(
    language="Spanish",
    name="Rodrigo",
    country_a="Mexico",
    country_b="Japan",
)

chat.predict_messages(prompt)

AIMessage(content='La distancia entre México y Japón es de aproximadamente 11,500 kilómetros. Mi nombre es Rodrigo. ¿En qué más puedo ayudarte?')